# Assignment 1



Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the above dataframe:

*   The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
*   Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
*   More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
*   If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
*   Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
*   In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [139]:
import pandas as pd # library for data analsysis
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from shapely.geometry import Point
from bs4 import BeautifulSoup
import xml
!pip install geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
!pip install folium pandas
import folium
from sklearn.cluster import KMeans
print("Libraries installed")

Libraries installed


*Scrape the List of postal codes of Canada*


In [105]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resource = requests.get(List_url).text

In [106]:
soup = BeautifulSoup(resource, 'xml')

In [107]:
table=soup.find('table')

In [108]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood
column_names = ['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns = column_names)

In [109]:
# Search all the postcode, borough, neighbourhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [110]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


*Time to clen data and remove rows were **Borough** is 'Not assigned'*

In [111]:
df=df[df['Borough']!='Not assigned']

In [112]:
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [113]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [114]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [115]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [116]:
df_merge.drop_duplicates(inplace=True)

In [117]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [118]:
df_merge.head()

,Postalcode,Borough,Neighbourhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [119]:
df_merge.shape

(103, 3)

# Assignment 2

**Geting latitude and logitude data**

Using file https://cocl.us/Geospatial_data to get the data.

Read the CSV data to a pandas dataframe.

In [120]:
geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [121]:
#exploring data
print("Geospatial dataframe's shape: ", geospatial_data.shape)
geospatial_data.head()

Geospatial dataframe's shape:  (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476



The size (103 rows) matches that of our original dataframe df.

**Combine two dataframes**

Time to combine two dataframes so that the latitude and logitude values from 'geospatial_data' are assigned to their corresponding rows in our dataframe, 'df'.

In [122]:
# Concatenate the two dataframes using column "PostalCode" from 'df' and
# "Postal Code" from 'geospatial_data'.
df = pd.concat(
    [df.set_index('Postalcode'), geospatial_data.set_index('Postal Code')],
    axis=1, join='inner')
df.head()

,Borough,Neighbourhood,Latitude,Longitude
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [123]:
#Postal code has been set as the index, change it to a non-index column and reset index.
df.reset_index(inplace=True)
df.head()

,index,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [124]:
#Rename the 'index' column to 'PostalCode'
df.rename(columns={'index': 'PostalCode'}, inplace=True)
df.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [125]:
print("Dataframe shape: ", df.shape)

Dataframe shape:  (103, 5)


#Assignment 3

Explore and cluster the neighborhoods in Toronto

In [126]:
#adding coordinates
df['Coordinates'] = list(zip(df['Latitude'], df['Longitude']))

In [127]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Coordinates
0,M3A,North York,Parkwoods,43.753259,-79.329656,"(43.7532586, -79.3296565)"
1,M4A,North York,Victoria Village,43.725882,-79.315572,"(43.725882299999995, -79.31557159999998)"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"(43.6542599, -79.3606359)"
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"(43.718517999999996, -79.46476329999999)"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"(43.6623015, -79.3894938)"


In [128]:
df['Coordinates'] = df['Coordinates'].apply(Point)

In [129]:
gdf = gpd.GeoDataFrame(df, geometry = 'Coordinates')

In [130]:
#droping Latitude and Longitude columns
gdf.drop(['Latitude','Longitude'], axis=1)

,PostalCode,Borough,Neighbourhood,Coordinates
0,M3A,North York,Parkwoods,POINT (43.75326 -79.32966)
1,M4A,North York,Victoria Village,POINT (43.72588 -79.31557)
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",POINT (43.65426 -79.36064)
3,M6A,North York,"Lawrence Manor, Lawrence Heights",POINT (43.71852 -79.46476)
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",POINT (43.66230 -79.38949)
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",POINT (43.66786 -79.53224)
6,M1B,Scarborough,"Malvern, Rouge",POINT (43.80669 -79.19435)
7,M3B,North York,Don Mills,POINT (43.74591 -79.35219)
8,M4B,East York,"Parkview Hill, Woodbine Gardens",POINT (43.70640 -79.30994)
9,M5B,Downtown Toronto,"Garden District, Ryerson",POINT (43.65716 -79.37894)


In [131]:
#working with only boroughs that contain the word Toronto 
gdf = gdf[gdf.Borough.str.contains("Toronto")]

In [132]:
gdf.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Coordinates
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,POINT (43.65426 -79.36064)
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,POINT (43.66230 -79.38949)
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,POINT (43.65716 -79.37894)
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,POINT (43.65149 -79.37542)
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,POINT (43.67636 -79.29303)


In [142]:

toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = df['Latitude']
Y = df['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map